In [ ]:
!pip uninstall -y transformers
!pip install -q ../input/nbme-git/transformers-4.19.0.dev0-py3-none-any.whl
!cp ../input/nbme-git/nbme/src/* ./
!ls

In [ ]:
from data_utils import NBMEDatasetInfer, preprocess_features, prepare_input
from model_utils import NBMEModel
from transformers import AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from eval_utils import get_char_logits, get_predictions, my_get_results
import os
import itertools
torch.set_grad_enabled(False)
DEBUG = False
BS = 4

# # blend-2 0.8937
# CHECKPOINTS = [
#     '../input/hf-models/facebook_bart-large',
#     '../input/huggingface-deberta-variants/deberta-base/deberta-base',
#     '../input/roberta-transformers-pytorch/roberta-base',
#     '../input/roberta-transformers-pytorch/roberta-large',
#     '../input/hf-models/facebook_muppet-roberta-large',
# ]
# PATHS = [
#     '../input/3a81-bart-large',
#     '../input/pl-3a81-deberta-base',
#     '../input/pl-3a81-roberta-base',
#     '../input/pl-b7ce-roberta-large',
#     '../input/pl-b7ce-muppet-roberta-large',
# ]
# FIXOFFS = [
#     False,
#     False,
#     False,
#     False,
#     False,
# ]
# WEIGHTS = {
#     'w0': 0.2610187004907005, 
#     'w1': 0.97620912150936, 
#     'w2': 0.3940880203235138, 
#     'w3': 0.4616469943029971, 
#     'w4': 0.8740497164412215,
# }

# # blend-3 0.8940
# CHECKPOINTS = [
#     '../input/hf-models/facebook_bart-large',
#     '../input/huggingface-deberta-variants/deberta-base/deberta-base',
#     '../input/roberta-transformers-pytorch/roberta-base',
#     '../input/hf-models/facebook_bart-large-mnli/',
#     '../input/roberta-transformers-pytorch/roberta-large',
#     '../input/hf-models/facebook_muppet-roberta-large',
# ]
# PATHS = [
#     '../input/3a81-bart-large',
#     '../input/pl-3a81-deberta-base',
#     '../input/pl-3a81-roberta-base',
#     '../input/pl-3a81-bart-large-mnli',
#     '../input/pl-b7ce-roberta-large',
#     '../input/pl-b7ce-muppet-roberta-large',
# ]
# FIXOFFS = [
#     False,
#     False,
#     False,
#     False,
#     False,
#     False,
# ]
# WEIGHTS = {
#     'w0': 0.3290705659298224, 
#     'w1': 0.8982042839509254, 
#     'w2': 0.2806373725161175, 
#     'w3': 0.712216288704943, 
#     'w4': 0.8471567075865541, 
#     'w5': 0.08386150855273114,
# }

# blend-4 0.8961
# CHECKPOINTS = [
#     '../input/roberta-transformers-pytorch/roberta-large',
#     '../input/huggingface-deberta-variants/deberta-base/deberta-base',
#     '../input/hf-models/nghuyong_ernie-2.0-large-en',
#     '../input/huggingface-deberta-variants/deberta-large-mnli/deberta-large-mnli',
#     '../input/hf-models/microsoft_deberta-v3-large',
# ]
# PATHS = [
#     '../input/pl-b7ce-roberta-large',
#     '../input/pl-3a81-deberta-base',
#     '../input/pl-3a81-ernie2-large-en',
#     '../input/pl-3a81-deberta-large-mnli',
#     '../input/pl-3a81-deberta-v3-large',
# ]
# FIXOFFS = [
#     False,
#     False,
#     True,
#     False,
#     False,
# ]
# WEIGHTS = {
#     'w0': 0.33915131925799213, 
#     'w1': 0.6919711522755465, 
#     'w2': 0.6449311288255997, 
#     'w3': 0.30586354998287263, 
#     'w4': 0.8212312568166007
# }

# CHECKPOINTS = [
#     '../input/huggingface-deberta-variants/deberta-xlarge/deberta-xlarge',
# ]
# PATHS = [
#     '../input/pl-3a81-deberta-xlarge',
# ]
# FIXOFFS = [
#     False,
# ]
# WEIGHTS = {
#     'w0': 1, 
# }

# blend-14 0.8967
# CHECKPOINTS = [
#     '../input/huggingface-deberta-variants/deberta-large-mnli/deberta-large-mnli',
#     '../input/hf-models/microsoft_deberta-v3-large',
#     '../input/huggingface-deberta-variants/deberta-xlarge/deberta-xlarge',
#     '../input/deberta-v2-xlarge',
# ]
# PATHS = [
#     '../input/pl-3a81-deberta-large-mnli',
#     '../input/pl-3a81-deberta-v3-large',
#     '../input/pl-3a81-deberta-xlarge',
#     '../input/pl-3a81-deberta-v2-xlarge-mnli',
# ]
# FIXOFFS = [
#     False,
#     False,
#     False,
#     False,
# ]
# WEIGHTS = {
#     'w0': 0.24077767284607712, 
#     'w1': 0.6090344856736725, 
#     'w2': 0.9677762586972335, 
#     'w3': 0.18805287188563702
# }
# WEIGHTS = {
#     'w0': 1, 
#     'w1': 1, 
#     'w2': 1, 
#     'w3': 1
# }

# blend-20 0.8967
CHECKPOINTS = [
    '../input/hf-models/microsoft_deberta-v3-large',
    '../input/huggingface-deberta-variants/deberta-xlarge/deberta-xlarge',
    '../input/deberta-v2-xlarge',
]
PATHS = [
    '../input/pl-3a81-deberta-v3-large',
    '../input/pl-3a81-deberta-xlarge',
    '../input/pl-3a81-deberta-v2-xlarge-mnli',
]
FIXOFFS = [
    False,
    False,
    False,
]
WEIGHTS = {'w0': 0.569056721365786, 
           'w1': 0.93844655010746, 
           'w2': 0.200064955738199}

In [ ]:
if DEBUG:
    test_df = pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')
else:
    test_df = pd.read_csv('../input/nbme-score-clinical-patient-notes/test.csv')
features = preprocess_features(pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv'))
pn = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')
test_df = test_df.merge(pn, on='pn_num', how='left')
test_df = test_df.merge(features, on='feature_num', how='left')
test_df['len'] = test_df['pn_history'].apply(len) + test_df['feature_text'].apply(len)
test_df = test_df.sort_values(by=['len']).reset_index(drop=True)
test_df

In [ ]:
char_logits_blend = [np.zeros(len(text)) for text in test_df.pn_history.values]
for i, ckpt in enumerate(CHECKPOINTS):
    model_path = PATHS[i]
    w = WEIGHTS[f'w{i}']
    print(f'{model_path} - weight = {w}')
    tokenizer = AutoTokenizer.from_pretrained(ckpt, trim_offsets=False)
    test_dataset = NBMEDatasetInfer(tokenizer, test_df)
    maxlen = max([len(x['input_ids']) for x in test_dataset])
    test_dataloader = DataLoader(test_dataset, batch_size=BS, shuffle=False, collate_fn=DataCollatorForTokenClassification(tokenizer), pin_memory=True)
    model = NBMEModel(ckpt).cuda()
    preds_folds = []
    if DEBUG:
        s, e = 0, 1
    else:
        s, e = 0, 5
    for fold in range(s, e):
        model.load_state_dict(torch.load(os.path.join(model_path, f'{fold}.pt')))
        model.eval()
        preds = []
        for b in tqdm(test_dataloader, total=len(test_dataset)//BS+1):
            b = {k: v.cuda() for k, v in b.items()}
            pred = model(**b).logits #[bs, maxlen, 1]
            pred = pred.view(pred.shape[0], pred.shape[1]) #[bs, maxlen]
            pred = F.pad(input=pred, pad=(0, maxlen-pred.shape[1]), mode='constant', value=-100).cpu().numpy()
            preds.append(pred)
        preds = np.concatenate(preds, axis=0) #[n, maxlen]
        preds_folds.append(preds)
    preds_folds = np.stack(preds_folds)
    print('preds_folds shape:', preds_folds.shape)
    preds = np.mean(preds_folds, axis=0)
    char_logits = get_char_logits(test_df['pn_history'].values, preds, tokenizer, do_fix_offsets=FIXOFFS[i])
    for j in range(len(test_df)):
        char_logits_blend[j] += w * char_logits[j]

In [ ]:
results = my_get_results(char_logits_blend, test_df.pn_history.values, th=0)
test_df['location'] = results
sub = pd.read_csv("../input/nbme-score-clinical-patient-notes/sample_submission.csv")
sub = sub[['id']].merge(test_df[['id', "location"]], how="left", on="id")
sub.to_csv('submission.csv', index=False)
sub